In [1]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt

In [2]:
model = tf.keras.models.load_model('food.model')

In [15]:
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [16]:
trainAug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest", validation_split=0.2)

mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean

In [21]:
trainGen = trainAug.flow_from_directory(
	'/home/karandeep/DL_data/violence_dataset',
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=True,
	batch_size=32, subset='training')

valGen = trainAug.flow_from_directory(
	'/home/karandeep/DL_data/violence_dataset',
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=32, subset='validation')

Found 8851 images belonging to 2 classes.
Found 2212 images belonging to 2 classes.


In [32]:
list(valGen.class_indices.keys())[0]

'non_violence'

In [33]:
from collections import deque
mean = np.array([123.68, 116.779, 103.939][::1], dtype="float32")
Q = deque(maxlen=128)

In [45]:
vs = cv2.VideoCapture('/home/karandeep/DL_data/archive/real life violence situations/Real Life Violence Dataset/Violence/V_300.mp4')
writer = None
(W, H) = (None, None)
# loop over frames from the video file stream
while True:
    # read the next frame from the file
    (grabbed, frame) = vs.read()
    # if the frame was not grabbed, then we have reached the end
    # of the stream
    if not grabbed:
        break
    # if the frame dimensions are empty, grab them
    if W is None or H is None:
        (H, W) = frame.shape[:2]
        
    output = frame.copy()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    frame = cv2.resize(frame, (224, 224)).astype("float32")
    frame -= mean
    preds = model.predict(np.expand_dims(frame, axis=0))[0]
    Q.append(preds)
    # perform prediction averaging over the current history of
    # previous predictions
    results = np.array(Q).mean(axis=0)
    i = np.argmax(results)
    label = list(valGen.class_indices.keys())[i]
    # draw the activity on the output frame
    text = "activity: {}".format(label)
    cv2.putText(output, text, (35, 50), cv2.FONT_HERSHEY_SIMPLEX,
                1.25, (0, 255, 0), 5)
    # check if the video writer is None
    if writer is None:
        # initialize our video writer
        fourcc = cv2.VideoWriter_fourcc(*"MJPG")
        writer = cv2.VideoWriter('non_violence.avi', fourcc, 30,
                                 (W, H), True)
        # write the output frame to disk
    writer.write(output)
    # show the output image
    cv2.imshow("Output", output)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
        break
# release the file pointers
print("[INFO] cleaning up...")
writer.release()
vs.release()
cv2.destroyAllWindows()

[INFO] cleaning up...


In [39]:
np.array(Q).mean(axis=0)

array([0.60987467, 0.39012533], dtype=float32)

In [41]:
i = np.argmax(results)
i

0

In [42]:
label

'non_violence'